In [82]:
import pandas as pd

In [83]:
df=pd.read_csv("train.csv")

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [84]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [85]:
df.shape

(20800, 5)

In [86]:
df=df.dropna()

In [87]:
df.shape

(18285, 5)

In [88]:
X=df.drop("label",axis=1)

In [89]:
y=df["label"]

In [90]:
X.shape

(18285, 4)

In [91]:
y.shape

(18285,)

In [92]:
import tensorflow as tf

In [16]:
tf.__version__

'2.8.0'

In [93]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [94]:
voc_size=5000

In [95]:
 messages=X.copy()

In [96]:
messages.reset_index(inplace=True)

In [97]:
import nltk
import re
from nltk.corpus import stopwords

In [98]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [99]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [37]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [38]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1283, 2943, 2570, 3537, 4892, 4822, 3392, 1204, 3174, 2532],
 [1632, 899, 2653, 2188, 1627, 4925, 2048],
 [3981, 3638, 2292, 1161],
 [389, 2115, 3324, 397, 3570, 584],
 [2249, 1627, 3608, 3411, 3546, 1849, 1627, 497, 1709, 3898],
 [4592,
  931,
  2763,
  3857,
  4517,
  3865,
  1081,
  2632,
  2078,
  4831,
  970,
  4137,
  3038,
  4715,
  2048],
 [1515, 3234, 1610, 1930, 1354, 395, 4004, 3043, 978, 63, 1094],
 [149, 1582, 3025, 4093, 3718, 2428, 3865, 3439, 978, 63, 1094],
 [929, 4880, 4843, 2949, 4899, 1778, 3125, 406, 3865, 972],
 [725, 4273, 3607, 381, 1355, 4698, 2945, 4973],
 [3037, 2732, 216, 3380, 2496, 4809, 1032, 3274, 4472, 2181, 4192],
 [397, 2125, 4892, 1778, 3865, 3718],
 [3564, 4949, 3385, 2004, 1591, 2619, 1192, 4385, 3291],
 [1232, 4799, 2662, 4801, 2550, 1637, 4802, 978, 63, 1094],
 [141, 1910, 4494, 2764, 3535, 978, 63, 1094],
 [689, 3822, 4969, 3674, 4149, 2487, 3, 2190, 1733, 4613],
 [3529, 2712, 899],
 [4432, 3519, 478, 191, 3865, 1972, 492, 2048],
 [1146, 3700,

In [100]:
sen_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sen_length)
print(embedded_docs)

[[   0    0    0 ... 1204 3174 2532]
 [   0    0    0 ... 1627 4925 2048]
 [   0    0    0 ... 3638 2292 1161]
 ...
 [   0    0    0 ...  978   63 1094]
 [   0    0    0 ... 1393 1106 4729]
 [   0    0    0 ... 3455 4744 4718]]


In [101]:
print(len(embedded_docs))

18285


## Creating Model

In [102]:
embedding_vector_features=40

In [103]:
from tensorflow.keras.layers import Dropout
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sen_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_2 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [104]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [105]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [106]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

## Model Training

In [107]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 18ms/step - loss: 0.3431 - accuracy: 0.8327 - val_loss: 0.1950 - val_accuracy: 0.9191
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1472 - accuracy: 0.9416 - val_loss: 0.1868 - val_accuracy: 0.9210
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1068 - accuracy: 0.9615 - val_loss: 0.1973 - val_accuracy: 0.9226
Epoch 4/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0832 - accuracy: 0.9702 - val_loss: 0.2301 - val_accuracy: 0.9215
Epoch 5/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0596 - accuracy: 0.9798 - val_loss: 0.2616 - val_accuracy: 0.9168
Epoch 6/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0448 - accuracy: 0.9851 - val_loss: 0.2813 - val_accuracy: 0.9168
Epoch 7/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0351 - accuracy: 0.9882 - val_loss: 0.3681 - val_accuracy: 0.9168

In [113]:
y_pred=model.predict(X_test)

In [61]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [114]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

In [115]:
confusion_matrix(y_test,y_pred)

array([[3098,  321],
       [ 178, 2438]], dtype=int64)

In [116]:
precision_score(y_test,y_pred)

0.883653497644074

In [117]:
accuracy_score(y_test,y_pred)

0.9173156586578294